In [1]:
import numpy as np
import pandas as pd
import requests
import xlsxwriter
import math
from scipy import stats
import yfinance as yf

In [96]:
data = pd.read_excel('stocks_info.xlsx')
data = data.iloc[:1210]
data

,代號,名稱,年月日,未調整收盤價(元),日報酬率 %,近一週報酬率 %,近一月報酬率 %,近一季報酬率 %,近一年報酬率 %
0,0050,元大台灣50,2023/12/29,135.45,-0.2210,1.8804,3.2789,10.4365,28.2125
1,0051,元大中型100,2023/12/29,73.65,0.2041,2.0790,5.0642,9.3610,49.3807
2,0052,富邦科技,2023/12/29,129.40,0.0000,2.4951,5.5464,14.1096,46.1906
3,0053,元大電子,2023/12/29,71.65,-0.4861,1.9204,3.6152,11.4158,42.3113
4,0055,元大MSCI金融,2023/12/29,24.12,-0.0414,1.8582,1.2596,5.7969,15.8559
...,...,...,...,...,...,...,...,...,...
1156,9944,新麗,2023/12/29,20.20,0.2481,0.0000,-0.9803,3.8560,3.5240
1157,9945,潤泰新,2023/12/29,37.75,-0.2642,4.1379,5.2998,7.2444,-17.9277
1158,9946,三發地產,2023/12/29,20.40,0.9901,0.4927,-0.9708,35.0992,73.1229
1159,9955,佳龍,2023/12/29,24.40,-0.4082,1.0352,-3.9370,13.4883,22.6126


In [97]:
my_columns = ['Ticker', 'Name','Price', 'One-Year Price Return', 'Number of Shares to Buy']

rows_list = []
for index, row in data.iterrows():
    symbol = row['代號']  # 访问股票代碼列
    name = row['名稱']    # 访问股票名稱列
    price = row['未調整收盤價(元)']
    year1ChangePercent = row['近一年報酬率 %']
    
    
    new_list = [
        symbol,
        name,
        price,
        year1ChangePercent,
        'N/A'
    ]
    rows_list.append(new_list)

momentum = pd.DataFrame(rows_list, columns=my_columns)
momentum.head()

,Ticker,Name,Price,One-Year Price Return,Number of Shares to Buy
0,0050,元大台灣50,135.45,28.2125,N/A
1,0051,元大中型100,73.65,49.3807,N/A
2,0052,富邦科技,129.40,46.1906,N/A
3,0053,元大電子,71.65,42.3113,N/A
4,0055,元大MSCI金融,24.12,15.8559,N/A


In [98]:
momentum.sort_values(by='One-Year Price Return', ascending=False, inplace = True)

In [99]:
momentum = momentum[:50]
momentum.reset_index(inplace = True)
momentum

,index,Ticker,Name,Price,One-Year Price Return,Number of Shares to Buy
0,268,1519,華城,327.00,570.6509,N/A
1,921,6117,迎廣,76.70,457.8152,N/A
2,800,3715,定穎投控,86.50,444.2155,N/A
3,845,4763,材料-KY,798.00,399.8298,N/A
4,926,6139,亞翔,170.00,374.2599,N/A
5,783,3661,世芯-KY,3275.00,328.1607,N/A
6,1080,8210,勤誠,271.50,286.0354,N/A
7,729,3231,緯創,98.60,243.2912,N/A
8,958,6235,華孚,114.50,236.7898,N/A
9,469,2382,廣達,224.50,236.0778,N/A


In [100]:
def portfolio_input():
    global portfolio_size 
    portfolio_size = input("Enter the size of your portfolio : ")
    
    try:
        float(portfolio_size)
    except ValueError:
        print("That is not a number! Please try again!")
        portfolio_size = input("Enter the size of your portfolio : ")

portfolio_input()
print(portfolio_size)   

1000000


In [101]:
position_size = float(portfolio_size)/len(momentum.index)
for i in range(len(momentum)):
    momentum.loc[i, "Number of Shares to Buy"] = math.floor(position_size / momentum.loc[i, "Price"])
    
momentum

,index,Ticker,Name,Price,One-Year Price Return,Number of Shares to Buy
0,268,1519,華城,327.00,570.6509,61
1,921,6117,迎廣,76.70,457.8152,260
2,800,3715,定穎投控,86.50,444.2155,231
3,845,4763,材料-KY,798.00,399.8298,25
4,926,6139,亞翔,170.00,374.2599,117
5,783,3661,世芯-KY,3275.00,328.1607,6
6,1080,8210,勤誠,271.50,286.0354,73
7,729,3231,緯創,98.60,243.2912,202
8,958,6235,華孚,114.50,236.7898,174
9,469,2382,廣達,224.50,236.0778,89


### Building a Better Momentum Strategy


In [102]:
hqm_columns = [
    'Ticker', 
    'Price', 
    'Number of Shares to Buy', 
    'One-Year Price Return', 
    'One-Year Return Percentile',
    'Three-Month Price Return',
    'Three-Month Return Percentile',
    'One-Month Price Return',
    'One-Month Return Percentile',
    'One-Week Price Return',
    'One-Week Return Percentile',
    'HQM Score'
]

In [103]:
rows_list = []
for index, row in data.iterrows():
    symbol = row['代號']  # 访问股票代碼列
    name = row['名稱']    # 访问股票名稱列
    price = row['未調整收盤價(元)']
    year1ChangePercent = row['近一年報酬率 %']
    month3ChangePercent = row['近一季報酬率 %']
    month1ChangePercent = row['近一月報酬率 %']
    week1ChangePercent = row['近一週報酬率 %']
    
    
    new_list = [
        symbol, 
        price, 
        'N/A', 
        year1ChangePercent, 
        'N/A',
        month3ChangePercent,
        'N/A',
        month1ChangePercent,
        'N/A',
        week1ChangePercent,
        'N/A',
        'N/A'
    ]
    rows_list.append(new_list)

hqm = pd.DataFrame(rows_list, columns=hqm_columns)
hqm.head()

,Ticker,Price,Number of Shares to Buy,One-Year Price Return,One-Year Return Percentile,Three-Month Price Return,Three-Month Return Percentile,One-Month Price Return,One-Month Return Percentile,One-Week Price Return,One-Week Return Percentile,HQM Score
0,0050,135.45,N/A,28.2125,N/A,10.4365,N/A,3.2789,N/A,1.8804,N/A,N/A
1,0051,73.65,N/A,49.3807,N/A,9.3610,N/A,5.0642,N/A,2.0790,N/A,N/A
2,0052,129.40,N/A,46.1906,N/A,14.1096,N/A,5.5464,N/A,2.4951,N/A,N/A
3,0053,71.65,N/A,42.3113,N/A,11.4158,N/A,3.6152,N/A,1.9204,N/A,N/A
4,0055,24.12,N/A,15.8559,N/A,5.7969,N/A,1.2596,N/A,1.8582,N/A,N/A
